# BLIP Caption Reproduction (COCO)

This notebook reproduces the BLIP captioning evaluation on the COCO dataset using the official repository.


In [1]:
%%bash
cd /kaggle/working
if [ ! -d BLIP ]; then
  git clone https://github.com/salesforce/BLIP.git
fi
cd BLIP
pwd
ls -la

/kaggle/working/BLIP
total 7380
drwxr-xr-x 7 root root    4096 Oct 27 06:55 .
drwxr-xr-x 3 root root    4096 Oct 27 06:55 ..
-rw-r--r-- 1 root root 6707660 Oct 27 06:55 BLIP.gif
-rw-r--r-- 1 root root    5159 Oct 27 06:55 CODE_OF_CONDUCT.md
-rw-r--r-- 1 root root     140 Oct 27 06:55 CODEOWNERS
-rw-r--r-- 1 root root     350 Oct 27 06:55 cog.yaml
drwxr-xr-x 2 root root    4096 Oct 27 06:55 configs
drwxr-xr-x 2 root root    4096 Oct 27 06:55 data
-rw-r--r-- 1 root root  687080 Oct 27 06:55 demo.ipynb
-rw-r--r-- 1 root root    4249 Oct 27 06:55 eval_nocaps.py
-rw-r--r-- 1 root root    9531 Oct 27 06:55 eval_retrieval_video.py
drwxr-xr-x 8 root root    4096 Oct 27 06:55 .git
-rw-r--r-- 1 root root    1482 Oct 27 06:55 LICENSE.txt
drwxr-xr-x 2 root root    4096 Oct 27 06:55 models
-rw-r--r-- 1 root root    3796 Oct 27 06:55 predict.py
-rw-r--r-- 1 root root    6666 Oct 27 06:55 pretrain.py
-rw-r--r-- 1 root root    9544 Oct 27 06:55 README.md
-rw-r--r-- 1 root root      65 Oct 27 06:55 req

Cloning into 'BLIP'...


## Install dependencies


In [2]:
%%bash
set -euo pipefail
cd /kaggle/working/BLIP
pip install -q timm==0.4.12 fairscale==0.4.4 transformers==4.30.2 tokenizers==0.13.3
pip install -q pyyaml ruamel.yaml==0.17.21 tqdm
pip install -q pycocoevalcap || pip install -q git+https://github.com/salaniz/pycocoevalcap
python - <<'PY'
import torch, ruamel.yaml, yaml
print('CUDA:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('GPU:', torch.cuda.get_device_name(0))
print('ruamel.yaml:', ruamel.yaml.__version__)
print('PyYAML:', yaml.__version__)
PY

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.4/235.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 102.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 113.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
kaggle-environments 1.18.0 requires transformers>=4.33.1, but you have transformers 4.30.2 which is incompatible.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.30.2 which is incompatible.
libcugraph-cu12 25.6.0 requires libraft-cu12==25.6.*, but you have libraft-cu12 25.2.0 which is incompatible.
gradio 5.38.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.0a1 which is incompatible.
pylibcugraph-cu12 25.6.0 requires pylibraft-cu12==25.6.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 25.6.0 requires rmm-cu12==25.6.*, but you have rmm-cu12 25.2.0 which is incompatible.


## Provide ruamel_yaml shim


In [3]:
%%bash
cd /kaggle/working/BLIP
cat <<'PY' > ruamel_yaml.py
from ruamel import yaml as _yaml

__all__ = [name for name in dir(_yaml) if not name.startswith('_')]
globals().update({name: getattr(_yaml, name) for name in __all__})
PY

## Download caption checkpoint


In [4]:
%%bash
set -euo pipefail
cd /kaggle/working/BLIP
mkdir -p checkpoints
wget -nv -P checkpoints https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_caption_capfilt_large.pth

2025-10-27 06:57:50 URL:https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_caption_capfilt_large.pth [896081425/896081425] -> "checkpoints/model_base_caption_capfilt_large.pth" [1]


## Prepare COCO image directories


In [5]:
from pathlib import Path

train_dir = Path('/kaggle/input/coco2014/train2014/train2014')
val_dir = Path('/kaggle/input/coco2014/val2014/val2014')

def count_images(directory: Path):
    if not directory.exists():
        print(f'{directory} does not exist')
        return 0
    files = list(directory.glob('*.jpg'))
    print(f'{directory.name}: {len(files):,} images')
    print('Sample:', [p.name for p in files[:5]])
    return len(files)

train_total = count_images(train_dir)
val_total = count_images(val_dir)
print(f'Total images inspected: {train_total + val_total:,}')

train2014: 82,783 images
Sample: ['COCO_train2014_000000263229.jpg', 'COCO_train2014_000000381595.jpg', 'COCO_train2014_000000147733.jpg', 'COCO_train2014_000000559395.jpg', 'COCO_train2014_000000374072.jpg']
val2014: 40,504 images
Sample: ['COCO_val2014_000000540531.jpg', 'COCO_val2014_000000159103.jpg', 'COCO_val2014_000000155671.jpg', 'COCO_val2014_000000033441.jpg', 'COCO_val2014_000000014321.jpg']
Total images inspected: 123,287


In [6]:
import os
import json
import shutil
from pathlib import Path
from urllib.request import urlretrieve

src_train = Path("/kaggle/input/coco2014/train2014/train2014")
src_val = Path("/kaggle/input/coco2014/val2014/val2014")
dest_root = Path("/kaggle/working/coco_images")
dest_train = dest_root / "train2014"
dest_val = dest_root / "val2014"

# ensure annotation files available locally
ann_root = Path("/kaggle/working/annotation")
ann_root.mkdir(parents=True, exist_ok=True)
urls = {
    "val": "https://storage.googleapis.com/sfr-vision-language-research/datasets/coco_karpathy_val.json",
    "test": "https://storage.googleapis.com/sfr-vision-language-research/datasets/coco_karpathy_test.json",
}
for split, url in urls.items():
    target = ann_root / f"coco_karpathy_{split}.json"
    if not target.exists():
        urlretrieve(url, target)

# rebuild destination tree cleanly
if dest_root.exists():
    shutil.rmtree(dest_root)
dest_train.mkdir(parents=True, exist_ok=True)
dest_val.mkdir(parents=True, exist_ok=True)

def safe_symlink(src: Path, dst: Path):
    if not src.exists():
        return False
    if dst.exists():
        return True
    dst.symlink_to(src.resolve())
    return True

# expose train images (already named COCO_train2014_XXXXXX.jpg)
train_count = 0
for path in src_train.glob("*.jpg"):
    if safe_symlink(path, dest_train / path.name):
        train_count += 1

# collect images required for val/test splits
required = set()
for split in ("val", "test"):
    ann_path = ann_root / f"coco_karpathy_{split}.json"
    with ann_path.open('r', encoding='utf-8') as f:
        data = json.load(f)
    for item in data:
        required.add(item["image"].split('/')[1])  # e.g. COCO_val2014_000000184613.jpg

missing = []
linked = 0
for filename in sorted(required):
    candidates = [
        src_train / filename.replace("val2014", "train2014"),
        src_val / filename,
    ]
    for candidate in candidates:
        if safe_symlink(candidate, dest_val / filename):
            linked += 1
            break
    else:
        missing.append(filename)

print(f"Linked {train_count} train images.")
print(f"Linked {linked} val/test images.")
if missing:
    print("Missing files:", missing[:10])
    raise FileNotFoundError(f"{len(missing)} images could not be located in the provided datasets.")


Linked 82783 train images.
Linked 10000 val/test images.


In [7]:
!mkdir -p /kaggle/working/annotation/coco_gt
!cp /kaggle/input/coco2014/captions/annotations/captions_val2014.json /kaggle/working/annotation/coco_gt/
!cp /kaggle/input/coco2014/captions/annotations/captions_train2014.json /kaggle/working/annotation/coco_gt/

## Update caption evaluation config


In [8]:
from ruamel.yaml import YAML
from pathlib import Path

yaml = YAML()
cfg_path = Path('/kaggle/working/BLIP/configs/caption_coco.yaml')
data = yaml.load(cfg_path.read_text())

data['image_root'] = '/kaggle/working/coco_images'
data['ann_root'] = '/kaggle/working/annotation'
data['coco_gt_root'] = '/kaggle/working/annotation/coco_gt'
data['pretrained'] = 'checkpoints/model_base_caption_capfilt_large.pth'

with cfg_path.open('w', encoding='utf-8') as f:
    yaml.dump(data, f)

print('Updated config entries:')
for key in ('image_root', 'ann_root', 'coco_gt_root', 'pretrained'):
    print('{}: {}'.format(key, data[key]))

Updated config entries:
image_root: /kaggle/working/coco_images
ann_root: /kaggle/working/annotation
coco_gt_root: /kaggle/working/annotation/coco_gt
pretrained: checkpoints/model_base_caption_capfilt_large.pth


## Run COCO caption evaluation


In [9]:
from pathlib import Path
import json

utils_path = Path("/kaggle/working/BLIP/data/utils.py")
utils_src = utils_path.read_text()

patch = """def coco_caption_eval(coco_gt_root, results_file, split):
    urls = {'val':'https://storage.googleapis.com/sfr-vision-language-research/datasets/coco_karpathy_val_gt.json',
            'test':'https://storage.googleapis.com/sfr-vision-language-research/datasets/coco_karpathy_test_gt.json'}
    filenames = {'val':'coco_karpathy_val_gt.json','test':'coco_karpathy_test_gt.json'}

    download_url(urls[split],coco_gt_root)
    annotation_file = os.path.join(coco_gt_root,filenames[split])

    # Some hosted Karpathy JSON files are missing the optional COCO 'info' section.
    # Newer pycocotools assumes the key exists, so patch up the annotation in-place.
    with open(annotation_file, 'r') as f:
        annotation_data = json.load(f)
    if 'info' not in annotation_data:
        annotation_data['info'] = {"description": "Inserted to satisfy pycocotools"}
        with open(annotation_file, 'w') as f:
            json.dump(annotation_data, f)

    # create coco object and coco_result object
    coco = COCO(annotation_file)
"""

start = utils_src.index("def coco_caption_eval")
end = utils_src.index("# create coco object", start)
utils_src = utils_src[:start] + patch + utils_src[end:]

utils_path.write_text(utils_src)


3976

In [10]:
from pathlib import Path

path = Path("/kaggle/working/BLIP/data/utils.py")
text = path.read_text()
text = text.replace("    dist.barrier()\n",
                    "    if utils.is_dist_avail_and_initialized():\n        dist.barrier()\n")
path.write_text(text)
print("Patched data/utils.py")


Patched data/utils.py


In [11]:
from pathlib import Path

blip_path = Path("/kaggle/working/BLIP/models/blip.py")
text = blip_path.read_text()
needle = "        if not sample:\n            image_embeds = image_embeds.repeat_interleave(num_beams,dim=0)\n            \n"
if needle in text:
    text = text.replace(needle, "")
    blip_path.write_text(text)
    print("Patched blip.py")
else:
    print("Patch already applied")


Patched blip.py


In [12]:
%%bash
set -euxo pipefail
cd /kaggle/working/BLIP
mkdir -p /kaggle/working/annotation
mkdir -p output/caption_coco
unset RANK WORLD_SIZE LOCAL_RANK MASTER_ADDR MASTER_PORT
CUDA_VISIBLE_DEVICES=0 python train_caption.py --config ./configs/caption_coco.yaml --output_dir output/caption_coco --evaluate 2>&1 | tee output/caption_coco/stdout_stderr.log


Not using distributed mode
Creating captioning dataset
100%|██████████| 84.6M/84.6M [00:02<00:00, 32.6MB/s]
Creating model
load checkpoint from checkpoints/model_base_caption_capfilt_large.pth
Start training
2025-10-27 07:00:49.482697: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761548449.725855     113 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761548449.797511     113 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Caption generation:  [  0/157]  eta: 1:00:32    time: 23.1342  data: 1.1662  max mem: 2330
Caption generation:  [ 10/157]  eta: 0:18:11    time: 7.4259  data: 0.1063  max mem: 2330
Caption generation:  [ 20/157]  eta: 0:15:18    

+ cd /kaggle/working/BLIP
+ mkdir -p /kaggle/working/annotation
+ mkdir -p output/caption_coco
+ unset RANK WORLD_SIZE LOCAL_RANK MASTER_ADDR MASTER_PORT
+ CUDA_VISIBLE_DEVICES=0
+ python train_caption.py --config ./configs/caption_coco.yaml --output_dir output/caption_coco --evaluate
+ tee output/caption_coco/stdout_stderr.log


## Extract key metrics


In [13]:
from pathlib import Path
log = Path('/kaggle/working/BLIP/output/caption_coco/stdout_stderr.log')
if not log.exists():
    raise FileNotFoundError('No evaluation log found at ' + str(log))
lines = log.read_text(errors='ignore').splitlines()
print('Captured metrics:')
for line in lines:
    if any(metric in line for metric in ('CIDEr', 'Bleu_4', 'Bleu_3', 'Bleu_2', 'Bleu_1', 'METEOR', 'SPICE')):
        print(line)


Captured metrics:
SPICE evaluation took: 8.245 min
Bleu_1: 0.787
Bleu_2: 0.638
Bleu_3: 0.506
Bleu_4: 0.400
computing METEOR score...
METEOR: 0.309
computing CIDEr score...
CIDEr: 1.323
computing SPICE score...
SPICE: 0.238
Bleu_1: 0.787
Bleu_2: 0.638
Bleu_3: 0.506
Bleu_4: 0.400
METEOR: 0.309
CIDEr: 1.323
SPICE: 0.238
SPICE evaluation took: 7.918 min
Bleu_1: 0.789
Bleu_2: 0.637
Bleu_3: 0.505
Bleu_4: 0.397
computing METEOR score...
METEOR: 0.310
computing CIDEr score...
CIDEr: 1.333
computing SPICE score...
SPICE: 0.238
Bleu_1: 0.789
Bleu_2: 0.637
Bleu_3: 0.505
Bleu_4: 0.397
METEOR: 0.310
CIDEr: 1.333
SPICE: 0.238
